In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path
import sys
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')

from pickle import load

# from models import resnet
from models import transformer

from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras import backend as K

In [86]:
data = pd.read_csv("../data/csv/2022/May_1/5_1_data.csv", sep = ",")

In [87]:
data['race_id'] = data['race_id'].astype(str)
data['race_round'] = data['race_round'].astype(str)
#data['total_horse_number'] = data['total_horse_number'].astype(str)
data['order'] = data['order'].astype(str)
data['frame_number'] = data['frame_number'].astype(str)
data['horse_number'] = data['horse_number'].astype(str)

In [88]:
print(data.isnull().sum())

race_id               0
race_round            0
race_title            0
ground_condition      0
date                  0
                     ..
ground_type_芝_3       9
ground_type_障_3       9
horse_weight_dif_3    9
same_jockey_3         9
same_jockey           0
Length: 156, dtype: int64


In [89]:
data = data.fillna(data.mean())

In [90]:
labels = data[["race_id","race_title","odds", "order", "horse_number", "goal_time", "half_order", "last_time", "horse_weight"]]

In [91]:
data = data.drop(["horse_number", "half_order", "goal_time" ,"last_time", "horse_weight",  "frame_number"], axis = 1)
print(data.shape)#"horse_weight_dif",,"pop"


(85, 150)


In [92]:
# 列の順序変更   "阪神":'place_阪神',"東京":'place_東京',"福島":'place_福島',
data = data.rename(columns ={"中山":'place_中山',"新潟":'place_新潟',"小倉":'place_小倉',"札幌":'place_札幌',"函館":'place_函館',"京都":'place_京都',"中京":'place_中京'})
data = data[['race_id','order',"race_round","ground_condition","total_horse_number","age","burden_weight","odds","race_rank","distance","ground_type_ダ","ground_type_芝","circle_右","circle_左","weather_circumstance_小雨","weather_circumstance_小雪","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_雨","weather_circumstance_雪","place_中京","place_中山","place_京都","place_函館","place_小倉","place_新潟","place_札幌","place_東京","place_福島","place_阪神","sex_セ","sex_牝","sex_牡","f_grass_win_rate","f_dart_win_rate","f_win_rate","g_f_grass_win_rate","g_f_dart_win_rate","g_f_win_rate","m_grass_win_rate","m_dart_win_rate","m_win_rate","same_jockey","whole_horse_number_1","odds_1","order_1","burden_weight_1","race_distance_1","ground_condition_1","goal_time_1","half_order_1","last_time_1","horse_weight_1","weather_circumstance_小雨_1","weather_circumstance_小雪_1","weather_circumstance_晴_1","weather_circumstance_曇_1","weather_circumstance_雨_1","weather_circumstance_雪_1","main_place_その他_1","main_place_中京_1","main_place_中山_1","main_place_京都_1","main_place_函館_1","main_place_小倉_1","main_place_新潟_1","main_place_札幌_1","main_place_東京_1","main_place_福島_1","main_place_阪神_1","race_rank_1","ground_type_ダ_1","ground_type_芝_1","ground_type_障_1","horse_weight_dif_1","same_jockey_1","whole_horse_number_2","odds_2","order_2","burden_weight_2","race_distance_2","ground_condition_2","goal_time_2","half_order_2","last_time_2","horse_weight_2","weather_circumstance_小雨_2","weather_circumstance_小雪_2","weather_circumstance_晴_2","weather_circumstance_曇_2","weather_circumstance_雨_2","weather_circumstance_雪_2","main_place_その他_2","main_place_中京_2","main_place_中山_2","main_place_京都_2","main_place_函館_2","main_place_小倉_2","main_place_新潟_2","main_place_札幌_2","main_place_東京_2","main_place_福島_2","main_place_阪神_2","race_rank_2","ground_type_ダ_2","ground_type_芝_2","ground_type_障_2","horse_weight_dif_2","same_jockey_2","whole_horse_number_3","odds_3","order_3","burden_weight_3","race_distance_3","ground_condition_3","goal_time_3","half_order_3","last_time_3","horse_weight_3","weather_circumstance_小雨_3","weather_circumstance_小雪_3","weather_circumstance_晴_3","weather_circumstance_曇_3","weather_circumstance_雨_3","weather_circumstance_雪_3","main_place_その他_3","main_place_中京_3","main_place_中山_3","main_place_京都_3","main_place_函館_3","main_place_小倉_3","main_place_新潟_3","main_place_札幌_3","main_place_東京_3","main_place_福島_3" ,"main_place_阪神_3","race_rank_3","ground_type_ダ_3","ground_type_芝_3","ground_type_障_3","horse_weight_dif_3","same_jockey_3"]]

In [93]:
# standard_scale = StandardScaler()
no_scale_data = data[['race_id','order']]
scale_columns = data.drop(['race_id','order'], axis=1).columns.values
standard_scale = load(open("../training/standard_scale.pkl", "rb"))
data = pd.DataFrame(standard_scale.fit_transform(data[scale_columns]))
data = pd.concat([data, no_scale_data], axis=1)
print(data.shape)

(85, 142)


In [84]:
def create_time_series_data(raw_data):
    number_of_race = raw_data.race_id.nunique()
    time_series_data = np.full((number_of_race, 18, 140), 0.0)#-float('inf')
#     label = np.full((number_of_race, 18), 19)
    race_number = 0
    horse_number = 0
    for i in range(len(raw_data)):
        if i == 0:
#             print(race_number)
#             print(horse_number)
#             print(raw_data.iloc[i].order)
#             label[race_number][horse_number] = float(raw_data.iloc[i].order)
            time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
            horse_number += 1
            continue
        if data.iloc[i].race_id != data.iloc[i-1].race_id:
            race_number += 1
            horse_number = 0
#             label[race_number][horse_number] = float(raw_data.iloc[i].order)
            time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
            horse_number += 1
        else:
#             label[race_number][horse_number] = float(raw_data.iloc[i].order)
            time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
            horse_number += 1
    return time_series_data

In [94]:
X = create_time_series_data(data)
print(X.shape)

(6, 18, 140)


In [15]:
X = X.astype('float32')
print(X.dtype)

float32


In [20]:
def categorical_focal_loss(alpha, gamma):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed

In [22]:
alpha = [0., 1.22620251, 1.21236789, 1.23692263, 1.24838235, 1.27347735, 1.27233213, 1.2771175, 1.33202573, 1.35824, 1.42768248, 1.52624955, 1.69170984, 1.90080609, 2.16445691, 2.54619076, 3.30697312, 14.51111111, 19.42562929, 0.13969294]

In [34]:
num_layers = 1 # hyperparameter
d_model = 140 # 4*35
num_heads = 20 # hyperparameter *must be a factor of d_model*
d_ffn = 8 # hyperparameter
pe_input = 18
target_size = 20
dropout_rate = 0.1 # hyperparameter

model1 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model1.compile(
    optimizer='Adam',
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model1.build(input_shape=(None, X.shape[1], X.shape[2]))
model1.load_weights("../models/results/transfomer1.h5")
pred1 = model1.predict(X)

In [37]:
num_layers = 2 # hyperparameter
d_model = 140 # 4*35
num_heads = 20 # hyperparameter *must be a factor of d_model*
d_ffn = 8 # hyperparameter
pe_input = 18
target_size = 20
dropout_rate = 0.1 # hyperparameter

model2 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model2.compile(
    optimizer='Adam',
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model2.build(input_shape=(None, X.shape[1], X.shape[2]))
model2.load_weights("../models/results/transfomer2.h5")
pred2 = model2.predict(X)

mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
final output shape (?, 18, 20)
mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
final output shape (?, 18, 20)


In [39]:
num_layers = 3 # hyperparameter
d_model = 140 # 4*35
num_heads = 20 # hyperparameter *must be a factor of d_model*
d_ffn = 8 # hyperparameter
pe_input = 18
target_size = 20
dropout_rate = 0.1 # hyperparameter

model3 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model3.compile(
    optimizer='Adam',
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model3.build(input_shape=(None, X.shape[1], X.shape[2]))
model3.load_weights("../models/results/transfomer3.h5")
pred3 = model3.predict(X)

mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
final output shape (?, 18, 20)
mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 

In [40]:
num_layers = 4 # hyperparameter
d_model = 140 # 4*35
num_heads = 20 # hyperparameter *must be a factor of d_model*
d_ffn = 8 # hyperparameter
pe_input = 18
target_size = 20
dropout_rate = 0.1 # hyperparameter

model4 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model4.compile(
    optimizer='Adam',
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model4.build(input_shape=(None, X.shape[1], X.shape[2]))
model4.load_weights("../models/results/transfomer4.h5")
pred4 = model4.predict(X)

mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
final output shape (?, 18, 20)
mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 

In [43]:
num_layers = 5 # hyperparameter
d_model = 140 # 4*35
num_heads = 20 # hyperparameter *must be a factor of d_model*
d_ffn = 8 # hyperparameter
pe_input = 18
target_size = 20
dropout_rate = 0.1 # hyperparameter

model5 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model5.compile(
    optimizer='Adam',
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model5.build(input_shape=(None, X.shape[1], X.shape[2]))
model5.load_weights("../models/results/transfomer5.h5")
pred5 = model5.predict(X)

mask shape after creating (?, 1, 1, 18)
mask shape after creating (?, 1, 1, 18)
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
(?, 18, 140) mha
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 18, 20, 7) split_head
(?, 20, 18, 7) q_shape
(?, 20, 18, 18) scaled_attention_shape
(?, 1, 1, 18) mask shape
final outp

In [44]:
# model6 = resnet.resrace(x.shape[1], 19)
# model6.load_weights("model/win5_resrace_model_best6.h5")
# pred6 = model6.predict(x)

# model7 = resnet.resrace(x.shape[1], 19)
# model7.load_weights("model/win5_resrace_model_best7.h5")
# pred7 = model7.predict(x)

# model8 = resnet.resrace(x.shape[1], 19)
# model8.load_weights("model/win5_resrace_model_best8.h5")
# pred8 = model8.predict(x)

# model9 = resnet.resrace(x.shape[1], 19)
# model9.load_weights("model/win5_resrace_model_best9.h5")
# pred9 = model9.predict(x)

# model10 = resnet.resrace(x.shape[1], 19)
# model10.load_weights("model/win5_resrace_model_best10.h5")
# pred10 = model10.predict(x)

In [67]:
log_pred1 = np.log(pred1)
log_pred2 = np.log(pred2)
log_pred3 = np.log(pred3)
log_pred4 = np.log(pred4)
log_pred5 = np.log(pred5)
# log_pred6 = np.log(pred6)
# log_pred7 = np.log(pred7)
# log_pred8 = np.log(pred8)
# log_pred9 = np.log(pred9)
# log_pred10 = np.log(pred10)

sum_pred = log_pred1 + log_pred2 + log_pred3 + log_pred4 + log_pred5 # + log_pred6 + log_pred7 + log_pred8 + log_pred9 + log_pred10
sum_pred = sum_pred.reshape([-1, 20])
print(sum_pred.shape)
pred_order = np.argmax(sum_pred, axis = -1)
print(pred_order.shape)

(108, 20)
(108,)


In [95]:
pred_df = pd.DataFrame(sum_pred, columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])
pred_order_df = pd.DataFrame(pred_order, columns = ["pred"])
summary = labels.join(pred_order_df)
summary = pd.concat([summary,pred_df], axis = 1)

In [99]:
print(summary.head(20))
print(pred_order_df.head(20))

            race_id race_title  odds order horse_number goal_time half_order  \
0   202203010610&rf      喜多方特別   2.8     ?            1         ?          ?   
1   202203010610&rf      喜多方特別   5.7     ?            2         ?          ?   
2   202203010610&rf      喜多方特別  37.1     ?            3         ?          ?   
3   202203010610&rf      喜多方特別  38.3     ?            4         ?          ?   
4   202203010610&rf      喜多方特別  12.3     ?            5         ?          ?   
5   202203010610&rf      喜多方特別   8.7     ?            6         ?          ?   
6   202203010610&rf      喜多方特別  26.0     ?            7         ?          ?   
7   202203010610&rf      喜多方特別  36.9     ?            8         ?          ?   
8   202203010610&rf      喜多方特別   0.0     ?            9         ?          ?   
9   202203010610&rf      喜多方特別  32.6     ?           10         ?          ?   
10  202203010610&rf      喜多方特別  10.7     ?           11         ?          ?   
11  202203010610&rf      喜多方特別  17.6    

In [71]:
summary[["race_title","odds","horse_number","pred",'1','2','3']].to_csv("../../weekly_prediction/2022/May_1/5_1_race_pred.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)